# Résolution Pb42 de la collection Hock-Schittkowski

Classification : QQR-T1-10

Number of variables : n = 4

Number of constraints : 2 (2 equalities)

# Modèle

$\left\{  
\begin{array}{l l l l} 
\min\ f(x) \\ 
\text{s.c.}\\
x_1-2 = 0\\
x_3^2+x_4^2-2= 0
\end{array} \right.$

where $f : x \longmapsto (x_1-1)^2+(x_2-2)^2+(x_3-3)^2+(x_4-4)^2$



Point de départ : $x_0 = (1,1,1,1)$

Solution attendue : $x^* = (2,2,0.6\sqrt 2, 0.8\sqrt 2)$

Valeur de la fonction objectif à la solution : $f(x^*) =28-10\sqrt 2$

In [ ]:
include("../src/enlsip_functions.jl")

In [ ]:
n = 4
m = 4
nb_eq = 2
nb_constraints = 2

In [ ]:
res42 = ResidualsEval(0)

function (res42::ResidualsEval)(x::Vector{Float64}, rx::Vector{Float64}, J::Matrix{Float64})

    # Evaluate the residuals
    if abs(res42.ctrl) == 1
        rx[:] = [x[1] - 1.0;
        x[2] - 2.0;
        x[3] - 3.0;
        x[4] - 4.0]

    # The jacobian is computed analytically
    elseif res42.ctrl == 2
        res42.ctrl = 0
    end
    return
end

cons42 = ConstraintsEval(0)

function (cons42::ConstraintsEval)(x::Vector{Float64}, cx::Vector{Float64}, A::Matrix{Float64})

    # Evaluate the constraints
    if abs(cons42.ctrl) == 1
        cx[:] = [x[1] - 2.0;
                 x[3]*x[3] + x[4]*x[4] - 2.0]

    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif cons42.ctrl == 2
        cons42.ctrl = 0
    end
    return
end

In [ ]:
include("../src/enlsip_functions.jl")
x0 = [1.0;1.0;1.0;1.0]
enlsip42 = enlsip(x0,res42,cons42,n,m,nb_eq,nb_constraints,MAX_ITER=8,scaling=true)

In [ ]:
x1 = enlsip42.sol[1]
x2 = enlsip42.sol[2]
x3 = enlsip42.sol[3]
x4 = enlsip42.sol[4]

println("Solution trouvée avec ENLSIP-Julia :")
@printf "x_jul = (%.9e, %.9e, %.9e, %.9e)\n" x1 x2 x3 x4
@printf "f(x_jul) = %.9e\n\n" enlsip42.obj_value 

println("\nValeurs théoriques visées :")
@printf "x_opt = (%.9e, %.9e, %.9e, %.9e)\n" 2.0 2.0 0.6*sqrt(2) 0.8*sqrt(2)
@printf "f(x_opt) = %.9e" 28 - 10*sqrt(2)

In [ ]:
using JuMP, Ipopt

In [ ]:
model = Model(with_optimizer(Ipopt.Optimizer))

@variable(model, x1, start = 1.0)
@variable(model, x2, start = 1.0)
@variable(model, x3, start = 1.0)
@variable(model, x4, start = 1.0)

In [ ]:
f(x1,x2,x3,x4) = (x1 - 1)^2 + (x2 - 2)^2 + (x3 - 3)^2 + (x4 - 4)^2
JuMP.register(model, :f, 4, f, autodiff=true)

In [ ]:
@NLconstraint(model, c1, x3^2 + x4^2 - 2.0 == 0)
@constraint(model, x1 - 2.0 == 0)

In [ ]:
@NLobjective(model, Min, f(x1,x2,x3,x4))
JuMP.optimize!(model)

In [ ]:
@printf "x_ipopt = (%.9e, %.9e, %.9e, %.9e)\n" JuMP.value(x1) JuMP.value(x2) JuMP.value(x3) JuMP.value(x4)